In [3]:
# [1] Install the latest version of transformers and accelerate directly from GitHub
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

# [2] Install Streamlit
!pip install streamlit

# [3] Install localtunnel using npm (for exposing localhost to web)
!npm install -g localtunnel


  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-4qqs90hu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-4qqs90hu
  Resolved https://github.com/huggingface/transformers.git to commit 0725cd6953803b8aacfc85288cbfb83dea30c469
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%%writefile CodeModel.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import time
def generate_code(prompt):
    model_id = "codellama/CodeLlama-7b-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    start = time.time()
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    sequences = pipeline(
            prompt,
            do_sample=True,
            temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=128,
            truncation=True
        )
    p = ''
    for seq in sequences:
        p = p + seq['generated_text']
    #print(p)
    end = time.time()
    return p, start, end


Writing CodeModel.py


In [5]:
%%writefile app.py
import streamlit as st
import time
from CodeModel import generate_code
def app():
    st.header("CodeGenie: AI-Powered Code Generation")
    user_input = st.text_area(
        "Enter your text prompt below and click the button to submit."
    )

    with st.form("my_form"):
        submit = st.form_submit_button(label="Submit text prompt")

    if submit:
        with st.spinner(text="Generating code ... It may take some time."):
            code, start, end = generate_code(prompt=user_input)
            hours, rem = divmod(end - start, 3600)
            minutes, seconds = divmod(rem, 60)

            st.success(
                "Processing time: {:0>2}:{:0>2}:{:05.2f}.".format(
                    int(hours), int(minutes), seconds
                )
            )

        st.write(code)

    st.sidebar.markdown("## Guide")
    st.sidebar.info("This tool uses Code Llama to generate code from the provided prompt. It supports many of the most popular languages being typed.")
    st.sidebar.markdown("## Examples")
    st.sidebar.write("1. Give a function definition to see how the AI completes it.")
    st.sidebar.write("2. Try different prompts!")
app()

Writing app.py


In [10]:
!streamlit run app.py --server.address=localhost --server.maxUploadSize=1000 --server.maxMessageSize=1000 &>/content/logs.txt &

In [11]:
import urllib
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Endpoint IP for localtunnel is: 34.125.93.57


In [12]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://big-pens-turn.loca.lt
^C
